In [149]:
import pandas as pd
import os
import boto3
import numpy as  np
import pandas as pd
import json
import time

In [2]:
# setup
os.chdir('//FS2\RTO\\CIS-PD MUSC\decoded_forms')
client = boto3.client('athena', 
                      region_name='us-west-2')
user_id_pairings = pd.read_csv("videoID.csv")
form509 = pd.read_hdf("form509.h5")
print(form509)

     SubjectCode  SiteID            VisitNm   FormDate  Q10  Q11  Q12  Q13  \
0         1003.0  1313.0    2 Weeks: Time 0 2017-07-03  2.0  0.0  0.0  1.0   
1         1003.0  1313.0   2 Weeks: Time 30 2017-07-03  1.0  0.0  0.0  0.0   
2         1003.0  1313.0   2 Weeks: Time 60 2017-07-03  1.0  2.0  0.0  0.0   
3         1003.0  1313.0   2 Weeks: Time 90 2017-07-03  0.0  0.0  0.0  0.0   
4         1003.0  1313.0  2 Weeks: Time 120 2017-07-03  1.0  0.0  0.0  1.0   
5         1003.0  1313.0  2 Weeks: Time 150 2017-07-03  1.0  1.0  0.0  0.0   
6         1003.0  1313.0            1 Month 2017-07-24  1.0  1.0  0.0  0.0   
7         1004.0  1332.0    2 Weeks: Time 0 2017-07-07  0.0  0.0  0.0  0.0   
8         1004.0  1332.0   2 Weeks: Time 30 2017-07-07  0.0  0.0  0.0  0.0   
9         1004.0  1332.0   2 Weeks: Time 60 2017-07-07  0.0  1.0  1.0  0.0   
10        1004.0  1332.0   2 Weeks: Time 90 2017-07-07  0.0  0.0  1.0  0.0   
11        1004.0  1332.0  2 Weeks: Time 120 2017-07-07  0.0  0.0

In [7]:
# get the 4 digit user_id from the corresponding 6 digit user_id
def user_id_6_to_4(user_id_6):
    for index, row in user_id_pairings.iterrows():
        if (row['Subj ID Athena']==user_id_6):
            return int(row['SubjectCode'])
    return None

In [8]:
# get the 6 digit user_id from the corresponding 4 digit user_id
def user_id_4_to_6(user_id_4):
    for index, row in user_id_pairings.iterrows():
        if (row['SubjectCode']==user_id_4):
            return int(row['Subj ID Athena'])
    return None

In [9]:
# generate query from row in form509
def generate_query(row):
    template1 = "SELECT *\nFROM watch_accelerometer\nwhere experiment = 'Clinicians'\nand user_id = "
    template2 = "\nand timestamp >= timestamp '"
    template3 = "'\nand timestamp <= timestamp '"
    template4 = "'"
    
    user_ID_4 = row['SubjectCode']
    
    user_ID_6 = user_id_4_to_6(user_ID_4).__str__()
    
    time = row['Q146_UTC']
    startTime =  (time - pd.Timedelta('00:15:00')).__str__()
    endTime = (time + pd.Timedelta('00:45:00')).__str__()
    
    if startTime == "NaT" or endTime == "NaT":
        print("invalid time")
        return None
    query = template1 + user_ID_6 + template2 + startTime + template3 + endTime + template4
    return query

In [10]:
# call query
def call_query(query):
    database='clinician_input_study'
    output_location='s3://aws-athena-query-results-arunjayaraman-us-west-2/'
    encryption_option='SSE_S3'

    query_info = client.start_query_execution(
        QueryString=query, 
        QueryExecutionContext={ 
            'Database': database
        },
        ResultConfiguration={ 
            'OutputLocation': output_location,
            'EncryptionConfiguration': { 
                'EncryptionOption': encryption_option
            }
        }
    )
    return query_info

In [11]:
paginator = client.get_paginator('get_query_results')

In [164]:
# return the results of the query associated with query_info
def get_query_results(query_info):
    query_id = query_info['QueryExecutionId']
    
    while True:
        try:
            page_iter = paginator.paginate(QueryExecutionId=query_id)
            results = []
            for page in page_iter:
                response = page['ResultSet']['Rows']
                results.append(response)
        except client.exceptions.InvalidRequestException:
            continue
        break
        
    results = list(np.concatenate(results))

#     while True:
#         try:
#             response = client.get_query_results(QueryExecutionId= query_id, MaxResults = 2000)
#         except client.exceptions.InvalidRequestException:
#             continue
#         break
    
#     results = pd.DataFrame(response)
#     print(response)
    
    # format the results
    if len(results)!=0:
        columns_names = [list(d.values())[0] for d in results[0]['Data']]

        D = pd.read_json(json.dumps(results[1::]))
        print(json.dumps(results[1::]))
        NewD = pd.DataFrame(index=D.index)
        for ind, col in zip(range(len(columns_names)),columns_names):
            NewD[col] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][ind], axis=1)
        return NewD
    else:
        return None

In [122]:
# get visit num from the visit information
def get_visit_num(VisitNm):
    if VisitNm.startswith("2 Weeks"):
        if VisitNm.endswith(" 0"):
            return '0'
        elif VisitNm.endswith("30"):
            return '1'
        elif VisitNm.endswith("60"):
            return '2'
        elif VisitNm.endswith("90"):
            return '3'
        elif VisitNm.endswith("120"):
            return '4'
        elif VisitNm.endswith("150"):
            return '5'
    else:
        return '6'

In [159]:
# fetch data for each row in form509
def iterate_over_form509():
    for index, row in form509.iloc[6:8,:].iterrows():
        t1 = time.time()
        id_4 = int(row['SubjectCode']).__str__()
        visit_num = get_visit_num(row['VisitNm'])
        path = "//FS2\RTO\\CIS-PD Study\MJFF Curation\ClinicVisitACC\\"
        filename = path + id_4 + "_session_" + visit_num + ".csv"
        
        query = generate_query(row)
        
        if (query is None):
            print("invalid query for subject "+ id_4 + " visit " + visit_num)
            continue
        
        query_info = call_query(query)
        query_results = get_query_results(query_info)
        
        if (query_results is None):
            print ("empty data for subject " + id_4 + " visit " + visit_num)
            continue
        print('done')
        t2 = time.time()
        print(t2-t1)
#         query_results.to_csv(filename)

            

In [162]:
# run
iterate_over_form509()

[]


ValueError: Wrong number of items passed 0, placement implies 1

In [145]:
results = r
columns_names = [list(d.values())[0] for d in results[0]['Data']]
    
D = pd.read_json(json.dumps(results[1::]))
NewD = pd.DataFrame(index=D.index)
for ind, col in zip(range(len(columns_names)),columns_names):  
    NewD[col] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][ind], axis=1)

In [146]:
NewD

,user_id,experiment,cohort,timestamp,x,y,z
0,142560,Clinicians,alabama,2017-07-03 15:20:00.380,0.85180664,0.4946289,-0.111572266
1,142560,Clinicians,alabama,2017-07-03 15:20:00.722,0.8647461,0.52075195,-0.099853516
2,142560,Clinicians,alabama,2017-07-03 15:20:00.440,0.84936523,0.548584,-0.10498047
3,142560,Clinicians,alabama,2017-07-03 15:20:00.037,0.8408203,0.5361328,-0.072509766
4,142560,Clinicians,alabama,2017-07-03 15:20:00.561,0.8581543,0.532959,-0.0949707
5,142560,Clinicians,alabama,2017-07-03 15:20:00.017,0.8442383,0.5378418,-0.09667969
6,142560,Clinicians,alabama,2017-07-03 15:20:00.702,0.86621094,0.52319336,-0.09692383
7,142560,Clinicians,alabama,2017-07-03 15:20:00.501,0.8598633,0.5305176,-0.092041016
8,142560,Clinicians,alabama,2017-07-03 15:20:00.843,0.8642578,0.52319336,-0.095458984
9,142560,Clinicians,alabama,2017-07-03 15:20:00.823,0.86572266,0.52075195,-0.091552734


In [144]:
columns_names

['user_id', 'experiment', 'cohort', 'timestamp', 'x', 'y', 'z']

In [113]:
D = pd.read_json(json.dumps(p['ResultSet']['Rows'][1::]))#.apply(lambda x: [list(d.values())[0] for d in x[0]])

NewD = pd.DataFrame(index=D.index)
NewD['user_id'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][0], axis=1)
NewD['experiment'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][1], axis=1)
NewD['cohort'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][2], axis=1)
NewD['timestamp'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][3], axis=1)
NewD['x'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][4], axis=1)
NewD['y'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][5], axis=1)
NewD['z'] = D.apply(lambda x: [list(d.values())[0] for d in x[0]][6], axis=1)

In [120]:
NewD

,user_id,experiment,cohort,timestamp,x,y,z
0,142560,Clinicians,alabama,2017-07-03 15:20:00.380,0.85180664,0.4946289,-0.111572266
1,142560,Clinicians,alabama,2017-07-03 15:20:00.722,0.8647461,0.52075195,-0.099853516
2,142560,Clinicians,alabama,2017-07-03 15:20:00.440,0.84936523,0.548584,-0.10498047
3,142560,Clinicians,alabama,2017-07-03 15:20:00.037,0.8408203,0.5361328,-0.072509766
4,142560,Clinicians,alabama,2017-07-03 15:20:00.561,0.8581543,0.532959,-0.0949707
5,142560,Clinicians,alabama,2017-07-03 15:20:00.017,0.8442383,0.5378418,-0.09667969
6,142560,Clinicians,alabama,2017-07-03 15:20:00.702,0.86621094,0.52319336,-0.09692383
7,142560,Clinicians,alabama,2017-07-03 15:20:00.501,0.8598633,0.5305176,-0.092041016
8,142560,Clinicians,alabama,2017-07-03 15:20:00.843,0.8642578,0.52319336,-0.095458984
9,142560,Clinicians,alabama,2017-07-03 15:20:00.823,0.86572266,0.52075195,-0.091552734


In [118]:
p['ResultSet']['Rows'][0]

{'Data': [{'VarCharValue': 'user_id'},
  {'VarCharValue': 'experiment'},
  {'VarCharValue': 'cohort'},
  {'VarCharValue': 'timestamp'},
  {'VarCharValue': 'x'},
  {'VarCharValue': 'y'},
  {'VarCharValue': 'z'}]}

In [103]:
[list(d.values())[0] for d in D.loc[0,:][0]]

['user_id', 'experiment', 'cohort', 'timestamp', 'x', 'y', 'z']

In [ ]:
dict.

In [50]:
r[0]

{'Data': [{'VarCharValue': 'user_id'},
  {'VarCharValue': 'experiment'},
  {'VarCharValue': 'cohort'},
  {'VarCharValue': 'timestamp'},
  {'VarCharValue': 'x'},
  {'VarCharValue': 'y'},
  {'VarCharValue': 'z'}]}

In [66]:
columns_names = pd.DataFrame(r[0]['Data'])
cols = columns_names.get_values()
columns = []
for col in cols:
    columns.append(col[0])

formatted_rows = []
formatted_results = pd.DataFrame(columns=columns)
for index, row in zip(range(len(r)),r):
    if index != 0:
        df = pd.DataFrame.from_dict(row['Data'])
        formatted_results = pd.concat([formatted_results,df])

formatted_results


KeyboardInterrupt: 

In [75]:
pd.read_json(json.dumps(r[0]))

,Data
0,{'VarCharValue': 'user_id'}
1,{'VarCharValue': 'experiment'}
2,{'VarCharValue': 'cohort'}
3,{'VarCharValue': 'timestamp'}
4,{'VarCharValue': 'x'}
5,{'VarCharValue': 'y'}
6,{'VarCharValue': 'z'}


In [72]:
str(r)

"[{'Data': [{'VarCharValue': 'user_id'}, {'VarCharValue': 'experiment'}, {'VarCharValue': 'cohort'}, {'VarCharValue': 'timestamp'}, {'VarCharValue': 'x'}, {'VarCharValue': 'y'}, {'VarCharValue': 'z'}]}\n {'Data': [{'VarCharValue': '142560'}, {'VarCharValue': 'Clinicians'}, {'VarCharValue': 'alabama'}, {'VarCharValue': '2017-07-03 15:20:00.380'}, {'VarCharValue': '0.85180664'}, {'VarCharValue': '0.4946289'}, {'VarCharValue': '-0.111572266'}]}\n {'Data': [{'VarCharValue': '142560'}, {'VarCharValue': 'Clinicians'}, {'VarCharValue': 'alabama'}, {'VarCharValue': '2017-07-03 15:20:00.722'}, {'VarCharValue': '0.8647461'}, {'VarCharValue': '0.52075195'}, {'VarCharValue': '-0.099853516'}]}\n ...\n {'Data': [{'VarCharValue': '142560'}, {'VarCharValue': 'Clinicians'}, {'VarCharValue': 'alabama'}, {'VarCharValue': '2017-07-03 16:03:51.119'}, {'VarCharValue': '0.62158203'}, {'VarCharValue': '0.61621094'}, {'VarCharValue': '0.5344238'}]}\n {'Data': [{'VarCharValue': '142560'}, {'VarCharValue': 'Clin

In [59]:
row[0]['Data']

[{'VarCharValue': '142560'},
 {'VarCharValue': 'Clinicians'},
 {'VarCharValue': 'alabama'},
 {'VarCharValue': '2017-07-03 15:20:00.722'},
 {'VarCharValue': '0.8647461'},
 {'VarCharValue': '0.52075195'},
 {'VarCharValue': '-0.099853516'}]

In [42]:
response['ResultSet']['Rows']

[{'Data': [{'VarCharValue': 'user_id'},
   {'VarCharValue': 'experiment'},
   {'VarCharValue': 'cohort'},
   {'VarCharValue': 'timestamp'},
   {'VarCharValue': 'x'},
   {'VarCharValue': 'y'},
   {'VarCharValue': 'z'}]},
 {'Data': [{'VarCharValue': '142560'},
   {'VarCharValue': 'Clinicians'},
   {'VarCharValue': 'alabama'},
   {'VarCharValue': '2017-07-03 15:20:00.380'},
   {'VarCharValue': '0.85180664'},
   {'VarCharValue': '0.4946289'},
   {'VarCharValue': '-0.111572266'}]},
 {'Data': [{'VarCharValue': '142560'},
   {'VarCharValue': 'Clinicians'},
   {'VarCharValue': 'alabama'},
   {'VarCharValue': '2017-07-03 15:20:00.722'},
   {'VarCharValue': '0.8647461'},
   {'VarCharValue': '0.52075195'},
   {'VarCharValue': '-0.099853516'}]},
 {'Data': [{'VarCharValue': '142560'},
   {'VarCharValue': 'Clinicians'},
   {'VarCharValue': 'alabama'},
   {'VarCharValue': '2017-07-03 15:20:00.440'},
   {'VarCharValue': '0.84936523'},
   {'VarCharValue': '0.548584'},
   {'VarCharValue': '-0.10498047'}

In [67]:
r[0]['Data']

[{'VarCharValue': 'user_id'},
 {'VarCharValue': 'experiment'},
 {'VarCharValue': 'cohort'},
 {'VarCharValue': 'timestamp'},
 {'VarCharValue': 'x'},
 {'VarCharValue': 'y'},
 {'VarCharValue': 'z'}]